In [1]:
with open("../input.txt") as f:
    input = [line.strip() for line in f.readlines()]
# print(input)

with open("sample-input.txt") as f:
    sample_input = [line.strip() for line in f.readlines()]
# print(f"Input: {sample_input}")

In [2]:
import numpy as np

graph_data = input

instructions = graph_data[0].strip()
graph = {}
for line in graph_data[2:]:
    if line.strip():
        node, children = line.strip().split(' = ')
        left, right = children.strip('()').split(', ')
        graph[node] = (left, right)

node_indices = {node: idx for idx, node in enumerate(graph.keys())}

start_nodes = np.zeros(len(node_indices))
end_nodes = np.zeros(len(node_indices))
left_connections = np.zeros((len(node_indices), len(node_indices)))
right_connections = np.zeros((len(node_indices), len(node_indices)))

for node, (left, right) in graph.items():
    if node.endswith('A'):
        start_nodes[node_indices[node]] = 1
    if node.endswith('Z'):
        end_nodes[node_indices[node]] = 1

    if left in node_indices:
        left_connections[node_indices[node], node_indices[left]] = 1
    if right in node_indices:
        right_connections[node_indices[node], node_indices[right]] = 1

instruction_matrix = {'L': np.array(left_connections), 'R': np.array(right_connections)}

def get_node_names(active_nodes):
    return [node for node, idx in node_indices.items() if active_nodes[idx].any() == 1]

In [ ]:
cycles = {}
for node, node_idx in node_indices.items():
    if not node.endswith('A'):
        continue
    state_vector = np.zeros_like(start_nodes)
    state_vector[node_idx] = 1

    step = 0
    
    while True:
        step += 1
        current_instruction = instructions[(step - 1) % len(instructions)]
        state_vector = state_vector.dot(instruction_matrix[current_instruction])

        if step % 10000 == 0:
            print(f"Step {step}, Nodes: {get_node_names(state_vector)}")

        if (state_vector.dot(end_nodes) == 1):
            print(f"Node: {node}, Total steps: {step}, Final nodes: {get_node_names(state_vector)}, end targets: {get_node_names(end_nodes)}")
            if node not in cycles:
                cycles[node] = [step]
            else:
                cycles[node].append(step)
            if len(cycles[node]) > 3:
                break

In [13]:
# Peek-a-boo, I see you
display(cycles)
display({node: [cycle[i] - cycle[i-1] for i in range(1, len(cycle))] for node, cycle in cycles.items()})

{'TSA': [16343, 32686, 49029, 65372, 81715, 98058],
 'JTA': [16897, 33794, 50691, 67588, 84485, 101382],
 'BLA': [21883, 43766, 65649, 87532, 109415, 131298],
 'NBA': [20221, 40442, 60663, 80884, 101105, 121326],
 'AAA': [19667, 39334, 59001, 78668, 98335, 118002],
 'QXA': [13019, 26038, 39057, 52076, 65095, 78114]}

{'TSA': [16343, 16343, 16343, 16343, 16343],
 'JTA': [16897, 16897, 16897, 16897, 16897],
 'BLA': [21883, 21883, 21883, 21883, 21883],
 'NBA': [20221, 20221, 20221, 20221, 20221],
 'AAA': [19667, 19667, 19667, 19667, 19667],
 'QXA': [13019, 13019, 13019, 13019, 13019]}

In [22]:
from functools import reduce
from math import gcd
reduce(lambda a,b: a * b // gcd(a, b), [x[0] for i, x in cycles.items()])

19185263738117